In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, sum as fsum, to_date, lit


spark = SparkSession.builder.getOrCreate()

In [0]:
dbutils.widgets.text("catalog", "new_york_taxi")            
dbutils.widgets.text("bronze_schema", "bronze")   
dbutils.widgets.text("silver_schema", "silver")    

CATALOG       = dbutils.widgets.get("catalog")
BRONZE_SCHEMA = dbutils.widgets.get("bronze_schema")
SILVER_SCHEMA = dbutils.widgets.get("silver_schema")

In [0]:
yellow_raw = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.yellow_tripdata")

In [0]:
# 2. Standartize the green trips
yellow_silver = (
    yellow_raw
    .select(
        lit("yellow").alias("service_type"),
        col("VendorID").alias("vendor_id"),
        col("tpep_pickup_datetime").alias("pickup_ts"),
        col("tpep_dropoff_datetime").alias("dropoff_ts"),
        lit(None).cast("int").alias("pickup_location_id"),   # not present in this file
        lit(None).cast("int").alias("dropoff_location_id"),  # not present in this file
        col("Passenger_Count").cast("int").alias("passenger_count"),
        col("Trip_Distance").cast("double").alias("trip_distance"),
        col("fare_amount").cast("double").alias("fare_amount"),
        col("tip_amount").cast("double").alias("tip_amount"),
        col("total_amount").cast("double").alias("total_amount"),
        col("payment_type").cast("string").alias("payment_type")
    )
    .filter(col("pickup_ts").isNotNull())
    .filter(col("dropoff_ts").isNotNull())
    .filter(col("trip_distance") >= 0)
    .withColumn("trip_date", to_date(col("pickup_ts")))
)

In [0]:
yellow_silver.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SILVER_SCHEMA}.trips_yellow")